# Synthetic Thread Generation
Using qim3d volume generation for creating a synthetic thread structure.

This notebook demonstrates a pipeline for creating realistic 3d synthetic threads using qim3d. The pipeline simulates realistic fiber structures by combining individual twisted threads into cohesibe rope volumes. 

The pipeline follows these main stpes:
1. **Single Thread Generation** - how to create individual thread volumes
2. **Object twisting** - How you can apply twists using qim3d
3. **Rope assembly** - How to combine multiple twisted threads into rope-like structures.

In [1]:
import numpy as np
from tqdm import tqdm
from typing import Tuple, List, Optional
import qim3d

## 1. Generating a single thread

Before creating complex rope structures, we start by understanding how to generate individual thread volumes. This section demonstrates the basic parameters and techniques for creating realistic fiber-like structures.


The thread generation uses several key parameters that control the appearance and structure:

- **length:** The longitudinal dimension of the thread (along the fiber axis)
- **thickness:** The cross-sectional diameter of the thread
- **noise_scale:** Controls surface texture and fiber irregularities (0.01-0.05 for realistic effects)
- **gamma:** Affects the sharpness of edges and density distribution (0.1-0.3 for fiber-like appearance)
- **threshold:** Determines the material density threshold (0.5-0.8 for solid fibers)
- **seed:** Random seed for reproducible results

In [2]:
length = 200
thickness = 15
noise_scale = 0.03  # Small noise for realistic texture
gamma = 0.1         # Sharp fiber edges
threshold = 0.6     # Dense material
seed = 42           # Reproducible results

thread_volume = qim3d.generate.volume(
    base_shape=(length, thickness, thickness),
    final_shape=(length, thickness, thickness),
    noise_scale=noise_scale,
    gamma=gamma,
    threshold=threshold,
    shape="cylinder",
    axis=0,
    seed=seed,
)

The generated thread can be visualized using qim3d's volumetric visualization to inspect the fiber structure and surface texture

In [3]:
qim3d.viz.volumetric(
    thread_volume
)

Output()

## 2. Apply thread twist
Real threads and fibers exhibit twist along their length, which is crucial for structural integrity and realistic appearance. This section demonstrates how to apply progressive twist transformations to simulate natural fiber behavior.


The qim3d's center_twist uses several key parameters:

- **twist_angle:** Total rotation angle in degrees (typical range: 90-360 degrees)
- **twist_axis:** Axis of rotation ("z" for longitudinal twist)
- **order:** Interpolation order (1 for linear)

In [4]:
twist_angle = 180
twist_axis = "z"

twisted_thread_volume = qim3d.operations.center_twist(
    thread_volume,
    rotation_angle=twist_angle,
    axis=twist_axis,
    order=1
)

In [5]:
qim3d.viz.volumetric(
    twisted_thread_volume,
)

Output()

## 3. Rope Assembly

To create more sophisticated rope structures, we need functions that can generate threads with integrated properties and handle complex assembly patterns.

### Thread Generation
This function creates threads with built-in twist and longitudinal variation for more realistic fiber appearance. The function works by using qim3d's generate volume and then applying center twist as shown above.

In [6]:
def generate_twisted_thread(length: int, thickness: int, twist_rate: float, 
                          phase_offset: float, noise_scale: float, seed: int) -> np.ndarray:
    """Generate a continuous thread with integrated twist."""
    
    # Generate base thread using qim3d's cylinder generation
    thread = qim3d.generate.volume(
        base_shape=(length, thickness, thickness),
        final_shape=(length, thickness, thickness),
        noise_scale=noise_scale,
        gamma=0.1,          # Sharp fiber edges
        threshold=0.6,      # Dense material
        max_value=240,      # High intensity for realistic appearance
        shape="cylinder",
        axis=0,             # Cylinder along length (z-axis)
        seed=seed
    )
    
    # Apply twist with phase offset
    total_rotation = twist_rate * 360 + phase_offset
    return qim3d.operations.center_twist(thread, rotation_angle=total_rotation, axis='z', order=1)


### Thread Integration
This function handles the process of integrating multiple 3d thread volumes while maintaining realistic physical relationships and preserving individual threads. It's needed as real fibers will compress and deform when pressed together rather than overlapping. The function works by: 
    
1. calculating thread positions with some natural waviness.
2. Detecting overlaps between threads.
3. Resolving overlaps using an intensity-based comparison.
4. Applying compression to create boundaries

In [7]:
def integrate_thread(combined_rope: np.ndarray, label_volume: np.ndarray, thread: np.ndarray,
                    rope_center_y: int, rope_center_x: int, radius: float, base_angle: float,
                    twist_rate: float, thread_id: int, compression_factor: float):
    """Integrate thread into rope volume with cohesive blending."""
    length, thread_thickness = thread.shape[0], thread.shape[1]
    half_thickness = thread_thickness // 2
    rope_shape = combined_rope.shape
    
    for z in range(length):
        z_progress = z / length
        rope_twist = twist_rate * 0.5 * 2 * np.pi * z_progress
        current_angle = base_angle + rope_twist
        
        # Calculate thread center with waviness
        center_y = rope_center_y + int(radius * np.cos(current_angle)) + int(2 * np.sin(z * 0.05 + base_angle))
        center_x = rope_center_x + int(radius * np.sin(current_angle)) + int(2 * np.cos(z * 0.07 + base_angle * 1.3))
        
        # Calculate placement bounds
        y_start, y_end = max(0, center_y - half_thickness), min(rope_shape[1], center_y + half_thickness)
        x_start, x_end = max(0, center_x - half_thickness), min(rope_shape[2], center_x + half_thickness)
        
        # Calculate thread slice bounds
        thread_y_start = max(0, half_thickness - (center_y - y_start))
        thread_y_end = min(thread_thickness, thread_y_start + (y_end - y_start))
        thread_x_start = max(0, half_thickness - (center_x - x_start))
        thread_x_end = min(thread_thickness, thread_x_start + (x_end - x_start))
        
        if not (y_end > y_start and x_end > x_start and thread_y_end > thread_y_start and thread_x_end > thread_x_start):
            continue
        
        # Extract sections
        thread_section = thread[z, thread_y_start:thread_y_end, thread_x_start:thread_x_end]
        thread_mask = thread_section > 0
        
        if not np.any(thread_mask):
            continue
        
        existing_rope = combined_rope[z, y_start:y_end, x_start:x_end]
        existing_labels = label_volume[z, y_start:y_end, x_start:x_end]
        
        # Handle placement
        overlap_mask = (existing_rope > 0) & thread_mask
        no_overlap_mask = (existing_rope == 0) & thread_mask
        
        new_intensity = existing_rope.astype(np.float32)
        new_labels = existing_labels.copy()
        
        # Place where no overlap
        new_intensity[no_overlap_mask] = thread_section[no_overlap_mask]
        new_labels[no_overlap_mask] = thread_id
        
        # Handle overlaps with intensity-based resolution
        if np.any(overlap_mask):
            existing_norm = existing_rope.astype(np.float32) / 255.0
            thread_norm = thread_section.astype(np.float32) / 255.0
            
            thread_wins = overlap_mask & (thread_norm >= existing_norm)
            existing_wins = overlap_mask & (thread_norm < existing_norm)
            
            compression_boost = 40
            
            if np.any(thread_wins):
                compressed = (thread_section[thread_wins].astype(np.float32) + 
                            existing_rope[thread_wins].astype(np.float32) * compression_factor + compression_boost)
                new_intensity[thread_wins] = np.minimum(255, compressed)
                new_labels[thread_wins] = thread_id
            
            if np.any(existing_wins):
                compressed = (existing_rope[existing_wins].astype(np.float32) + 
                            thread_section[existing_wins].astype(np.float32) * compression_factor + compression_boost * 0.5)
                new_intensity[existing_wins] = np.minimum(255, compressed)
        
        # Update rope volume
        combined_rope[z, y_start:y_end, x_start:x_end] = new_intensity.astype(np.uint8)
        label_volume[z, y_start:y_end, x_start:x_end] = new_labels

### Complete Rope Generation
This final function coordinates all of the aspects of the rope generation. The function works by:

1. **Calculating optimal concentric ring layouts** for thread positioning based on rope diameter and thread thickness
2. **Distributing threads across rings** using circumference constraints to determine how many threads fit per ring
3. **Generating individual threads** with coordinated twist phases and unique random seeds for variation
4. **Integrating threads into the rope volume** using the thread integration function to handle overlaps
5. **Applying post-processing smoothing** to create cohesive rope appearance while preserving thread boundaries
6. **Tracking metadata** for each thread including position, twist parameters, and generation settings

In [8]:
def generate_rope(
    num_threads: int = 6,
    rope_length: int = 300,
    rope_diameter: int = 60,
    thread_thickness: int = 12,
    twist_rate: float = 2.0,
    compression_factor: float = 0.8,
    thread_spacing: float = 0.9,
    noise_scale: float = 0.03,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray, List[dict]]:
    """Generate a cohesive solid rope where threads are tightly integrated."""
    np.random.seed(seed)
    
    rope_shape = (rope_length, rope_diameter, rope_diameter)
    combined_rope = np.zeros(rope_shape, dtype=np.uint8)
    label_volume = np.zeros(rope_shape, dtype=np.uint8)
    thread_metadata = []
    
    rope_center_y, rope_center_x = rope_shape[1] // 2, rope_shape[2] // 2
    max_radius = (rope_diameter - thread_thickness) // 2
    
    # Calculate thread positions in concentric rings
    positions = []
    ring_spacing = thread_thickness * thread_spacing
    num_rings = max(1, int(max_radius / ring_spacing))
    threads_placed = 0
    
    for ring_idx in range(num_rings):
        if threads_placed >= num_threads:
            break
            
        if ring_idx == 0:
            ring_radius, threads_in_ring = 0, min(1, num_threads)
        else:
            ring_radius = ring_idx * ring_spacing
            circumference = 2 * np.pi * ring_radius
            threads_in_ring = min(
                int(circumference / (thread_thickness * thread_spacing)),
                num_threads - threads_placed
            )
        
        for i in range(threads_in_ring):
            angle = 0 if threads_in_ring == 1 else (2 * np.pi * i) / threads_in_ring + (ring_idx % 2) * (np.pi / threads_in_ring)
            positions.append((ring_radius, angle, ring_idx))
            threads_placed += 1
    
    print(f"Generating cohesive solid rope with {min(num_threads, len(positions))} threads...")
    
    for thread_idx in tqdm(range(min(num_threads, len(positions))), desc="Generating solid rope threads"):
        radius, base_angle, ring_idx = positions[thread_idx]
        
        # Generate thread with integrated twist
        thread = generate_twisted_thread(rope_length, thread_thickness, twist_rate, 
                                       thread_idx * (360 / num_threads), noise_scale, seed + thread_idx)
        
        # Integrate thread into rope
        integrate_thread(combined_rope, label_volume, thread, rope_center_y, rope_center_x, 
                        radius, base_angle, twist_rate, thread_idx + 1, compression_factor)
        
        thread_metadata.append({
            'thread_id': thread_idx + 1, 'ring': ring_idx, 'radius': radius,
            'base_angle': base_angle, 'twist_rate': twist_rate,
            'compression_factor': compression_factor, 'thickness': thread_thickness,
            'seed': seed + thread_idx
        })
    
    # Apply final smoothing
    rope_mask = combined_rope > 0
    smoothed = qim3d.filters.gaussian(combined_rope.astype(np.float32), sigma=0.5)
    result = combined_rope.astype(np.float32)
    result[rope_mask] = 0.7 * combined_rope[rope_mask] + 0.3 * smoothed[rope_mask]
    
    return result.astype(np.uint8), label_volume, thread_metadata




Now we can use our complete pipeline to generate a realistic rope structure:

In [9]:
rope, labels, metadata = generate_rope(
    num_threads=18,
    rope_length=300,
    rope_diameter=80,
    thread_thickness=10,
    twist_rate=2,
    compression_factor=0,
    thread_spacing=0.8,
    seed=42
)

Generating cohesive solid rope with 18 threads...


Generating solid rope threads: 100%|██████████| 18/18 [00:01<00:00, 13.71it/s]


We can then visualize the generated rope and its labels using qim3d's visualization tools:

In [10]:
qim3d.viz.volumetric(
    rope
)

Output()

We can even use `viz.colormaps.segmentation()` which uses the thread labels to create a colour map, making it easier to visualise the thread boundaries more clearly.

In [11]:
# Visualize the generated rope with labels
num_threads = len(labels)
cmap = qim3d.viz.colormaps.segmentation(num_labels=num_threads)

qim3d.viz.slicer_orthogonal(
    labels,
    color_map=cmap,
    value_max=num_threads
)